In [1]:
using Perla1, DataFrames, DataFramesMeta, CSV, Parameters

In [2]:
using Plots
gr(fmt = :png)

Plots.GRBackend()

In [3]:
ϕ_default = 0.8
η_default = 6.9652841785386155
ν_default = 0.01948530463012872
ηs = range(5.0, 8.0, length = 40)
μ_i_bounds = (x, y, z) -> [1.0, Inf]
νs = range(0.01, 0.03, length = 40)
δ_Ms = range(0.0225, 0.2, length = 40)
settings = stationary_settings_default()
industry_settings = merge(stationary_settings_default(),
    industry_settings_default(
        μ_i_bounds = μ_i_bounds,
        ftol = 1e-4, 
        ignore_p_i_deviations = true, 
        trace_iterations = true))

function d_alt(θ_i, μ_i, params)
    @unpack η, ν, ϕ = params
    return (θ_i^ϕ/(ν^(ϕ/2)) + abs(μ_i-1.0)^ϕ/η)^(2/ϕ)
end

d_alt (generic function with 1 method)

# Results by `η`

In [4]:
θs = zeros(0)
μs = zeros(0)
Qs = zeros(0)
Bs = zeros(0)
profit_shares = zeros(0)

for η in ηs
    # fix other parameters other than δ_M
    params = stationary_params_default()
    industry_params = merge(industry_params_default(ν = ν_default, η = η, d = d_alt, ϕ = ϕ_default), params)
    @time sol = industry_equilibrium(industry_params, industry_settings)
    
    θ = sol.zero[1]
    μ = sol.zero[2]
    @time ss = steadystate(merge(params, (θ = θ, μ = μ)), settings)
    
    append!(θs, θ)
    append!(μs, μ)
    append!(Qs, ss.Q)
    append!(Bs, ss.B)
    append!(profit_shares, ss.profit_share)
end

 19.784320 seconds (63.97 M allocations: 3.182 GiB, 8.65% gc time)
  3.702319 seconds (11.81 M allocations: 608.514 MiB, 9.78% gc time)
  0.500811 seconds (941.35 k allocations: 130.970 MiB, 4.65% gc time)
  0.005883 seconds (37.42 k allocations: 3.975 MiB)
  0.389881 seconds (776.60 k allocations: 106.493 MiB, 3.34% gc time)
  0.004033 seconds (37.42 k allocations: 3.975 MiB)
  0.453864 seconds (871.21 k allocations: 120.520 MiB, 2.78% gc time)
  0.006871 seconds (37.72 k allocations: 4.006 MiB)
  0.398679 seconds (814.45 k allocations: 112.111 MiB, 3.49% gc time)
  0.004079 seconds (37.42 k allocations: 3.975 MiB)
  0.404647 seconds (814.77 k allocations: 112.229 MiB, 3.55% gc time)
  0.004186 seconds (37.42 k allocations: 3.975 MiB)
  0.553177 seconds (1.05 M allocations: 146.643 MiB, 2.62% gc time)
  0.007084 seconds (37.72 k allocations: 4.006 MiB, 31.48% gc time)
  0.490773 seconds (948.00 k allocations: 131.947 MiB, 2.56% gc time)
  0.004578 seconds (37.42 k allocations: 3.975 M

In [5]:
results_by_ηs = DataFrame(η = ηs,
    ν = industry_params_default().ν,
    δ_M = stationary_params_default().δ_M,
    θ = θs,
    μ = μs,
    Q = Qs, B = Bs, 
    profit_share = profit_shares)
results_by_ηs.change_by = "eta";
results_by_ηs

,η,ν,δ_M,θ,μ,Q,B,profit_share
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,5.0,0.0178923,0.056,0.0631714,1.0,0.761829,0.804039,0.195961
2,5.07692,0.0178923,0.056,0.0631757,1.0,0.761839,0.804042,0.195958
3,5.15385,0.0178923,0.056,0.0631713,1.0,0.761829,0.804039,0.195961
4,5.23077,0.0178923,0.056,0.063196,1.0,0.761884,0.804053,0.195947
5,5.30769,0.0178923,0.056,0.0631669,1.0,0.761819,0.804037,0.195963
6,5.38462,0.0178923,0.056,0.0631712,1.0,0.761829,0.804039,0.195961
7,5.46154,0.0178923,0.056,0.0631849,1.0,0.761859,0.804047,0.195953
8,5.53846,0.0178923,0.056,0.0631588,1.0,0.761801,0.804033,0.195967
9,5.61538,0.0178923,0.056,0.063183,1.0,0.761855,0.804046,0.195954


# Results by `ν`

In [6]:
θs = zeros(0)
μs = zeros(0)
Qs = zeros(0)
Bs = zeros(0)
profit_shares = zeros(0)

for ν in νs
    # fix other parameters other than ν
    params = stationary_params_default()
    industry_params = merge(industry_params_default(η = η_default, d = d_alt, ϕ = ϕ_default, ν = ν), params)
    @time sol = industry_equilibrium(industry_params, industry_settings)
    
    θ = sol.zero[1]
    μ = sol.zero[2]
    @time ss = steadystate(merge(params, (θ = θ, μ = μ)), settings)
    
    append!(θs, θ)
    append!(μs, μ)
    append!(Qs, ss.Q)
    append!(Bs, ss.B)
    append!(profit_shares, ss.profit_share)
end

  0.406724 seconds (903.14 k allocations: 119.974 MiB, 4.54% gc time)
  0.004099 seconds (35.93 k allocations: 3.817 MiB)
  0.346064 seconds (731.31 k allocations: 97.351 MiB, 3.75% gc time)
  0.004723 seconds (34.74 k allocations: 3.691 MiB)
  0.427096 seconds (953.52 k allocations: 127.217 MiB, 3.75% gc time)
  0.004475 seconds (35.93 k allocations: 3.817 MiB)
  0.446637 seconds (978.27 k allocations: 129.674 MiB, 4.68% gc time)
  0.003757 seconds (35.33 k allocations: 3.754 MiB)
  0.539921 seconds (1.04 M allocations: 139.446 MiB, 4.14% gc time)
  0.005078 seconds (35.63 k allocations: 3.786 MiB)
  0.465812 seconds (928.80 k allocations: 123.626 MiB, 3.92% gc time)
  0.008215 seconds (35.33 k allocations: 3.754 MiB, 40.56% gc time)
  0.542529 seconds (1.03 M allocations: 138.748 MiB, 3.37% gc time)
  0.004196 seconds (35.63 k allocations: 3.786 MiB)
  0.232338 seconds (527.88 k allocations: 68.836 MiB, 5.14% gc time)
  0.005199 seconds (35.93 k allocations: 3.817 MiB)
  0.296814 sec

In [7]:
results_by_νs = DataFrame(η = η_default,
    ν = νs,
    δ_M = stationary_params_default().δ_M,
    θ = θs,
    μ = μs,
    Q = Qs, B = Bs, 
    profit_share = profit_shares)
results_by_νs.change_by = "nu";
results_by_νs

,η,ν,δ_M,θ,μ,Q,B,profit_share
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,6.96528,0.01,0.056,0.0401047,1.39516,0.756637,0.817054,0.182946
2,6.96528,0.0105128,0.056,0.0412823,1.42581,0.762924,0.818918,0.181082
3,6.96528,0.0110256,0.056,0.0424321,1.45504,0.768806,0.82059,0.17941
4,6.96528,0.0115385,0.056,0.043574,1.4822,0.7743,0.822074,0.177926
5,6.96528,0.0120513,0.056,0.0447127,1.50777,0.779493,0.823416,0.176584
6,6.96528,0.0125641,0.056,0.0458303,1.5323,0.784412,0.824642,0.175358
7,6.96528,0.0130769,0.056,0.0469374,1.556,0.789118,0.825776,0.174224
8,6.96528,0.0135897,0.056,0.0480489,1.57816,0.793597,0.82681,0.17319
9,6.96528,0.0141026,0.056,0.0491206,1.60059,0.797914,0.827793,0.172207


# Results by `δ_M `

In [8]:
θs = zeros(0)
μs = zeros(0)
Qs = zeros(0)
Bs = zeros(0)
profit_shares = zeros(0)

for δ_M in δ_Ms
    # fix other parameters other than δ_M
    params = stationary_params_default(δ_M = δ_M)
    industry_params = merge(industry_params_default(η = η_default, d = d_alt, ϕ = ϕ_default), params)
    @time sol = industry_equilibrium(industry_params, industry_settings)
    
    θ = sol.zero[1]
    μ = sol.zero[2]
    ss = steadystate(merge(params, (θ = θ, μ = μ)), settings)
    
    append!(θs, θ)
    append!(μs, μ)
    append!(Qs, ss.Q)
    append!(Bs, ss.B)
    append!(profit_shares, ss.profit_share)
end

  1.019662 seconds (2.02 M allocations: 207.768 MiB, 4.21% gc time)
  0.409270 seconds (1.27 M allocations: 145.698 MiB, 5.37% gc time)
  0.339270 seconds (959.62 k allocations: 114.569 MiB, 5.58% gc time)
  0.418828 seconds (1.18 M allocations: 141.997 MiB, 4.58% gc time)
  0.346888 seconds (803.36 k allocations: 102.935 MiB, 4.07% gc time)
  0.251470 seconds (623.16 k allocations: 79.447 MiB, 4.36% gc time)
  0.193211 seconds (498.62 k allocations: 63.497 MiB, 5.32% gc time)
  0.180586 seconds (466.41 k allocations: 59.305 MiB, 4.42% gc time)
  0.176988 seconds (435.46 k allocations: 55.341 MiB, 3.36% gc time)
  0.164254 seconds (403.49 k allocations: 51.181 MiB, 6.25% gc time)
  0.164284 seconds (411.93 k allocations: 52.857 MiB, 3.50% gc time)
  0.194608 seconds (414.07 k allocations: 53.250 MiB, 5.18% gc time)
  0.181261 seconds (405.38 k allocations: 52.406 MiB, 3.57% gc time)
  0.185891 seconds (405.06 k allocations: 52.826 MiB, 2.77% gc time)
  0.212714 seconds (404.61 k alloca

In [9]:
results_by_δ_Ms = DataFrame(η = η_default,
    ν = industry_params_default().ν,
    δ_M = δ_Ms,
    θ = θs,
    μ = μs,
    Q = Qs, B = Bs, 
    profit_share = profit_shares)
results_by_δ_Ms.change_by = "delta_M";
results_by_δ_Ms

,η,ν,δ_M,θ,μ,Q,B,profit_share
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,6.96528,0.0178923,0.0225,0.0495398,1.64724,0.942113,0.841616,0.158384
2,6.96528,0.0178923,0.0270513,0.0511701,1.68215,0.921885,0.840048,0.159952
3,6.96528,0.0178923,0.0316026,0.0524421,1.70438,0.903183,0.838687,0.161313
4,6.96528,0.0178923,0.0361538,0.0534914,1.71931,0.885973,0.837489,0.162511
5,6.96528,0.0178923,0.0407051,0.0544053,1.72939,0.870183,0.836409,0.163591
6,6.96528,0.0178923,0.0452564,0.05521,1.73659,0.855665,0.835424,0.164576
7,6.96528,0.0178923,0.0498077,0.0559506,1.74145,0.842254,0.834516,0.165484
8,6.96528,0.0178923,0.054359,0.0566363,1.74472,0.829773,0.833672,0.166328
9,6.96528,0.0178923,0.0589103,0.0572738,1.74681,0.818066,0.832886,0.167114


# Write CSV

In [10]:
equilibriums = deepcopy(results_by_ηs)
append!(equilibriums, results_by_νs)
append!(equilibriums, results_by_δ_Ms)

# change unicode column names to English
rename!(equilibriums, :η => :eta)
rename!(equilibriums, :ν => :nu)
rename!(equilibriums, :δ_M => :delta_M)
rename!(equilibriums, :μ => :mu)
rename!(equilibriums, :θ => :theta)

CSV.write("endogenous_comparative_statics_phi_0.8.csv", equilibriums)
equilibriums

,eta,nu,delta_M,theta,mu,Q,B,profit_share
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,5.0,0.0178923,0.056,0.0631714,1.0,0.761829,0.804039,0.195961
2,5.07692,0.0178923,0.056,0.0631757,1.0,0.761839,0.804042,0.195958
3,5.15385,0.0178923,0.056,0.0631713,1.0,0.761829,0.804039,0.195961
4,5.23077,0.0178923,0.056,0.063196,1.0,0.761884,0.804053,0.195947
5,5.30769,0.0178923,0.056,0.0631669,1.0,0.761819,0.804037,0.195963
6,5.38462,0.0178923,0.056,0.0631712,1.0,0.761829,0.804039,0.195961
7,5.46154,0.0178923,0.056,0.0631849,1.0,0.761859,0.804047,0.195953
8,5.53846,0.0178923,0.056,0.0631588,1.0,0.761801,0.804033,0.195967
9,5.61538,0.0178923,0.056,0.063183,1.0,0.761855,0.804046,0.195954
